In [100]:
import pandas as pd 
import requests
import json
from datetime import datetime

In [101]:
JSON_URL = "https://wowtokenprices.com/history_prices_full.json"

In [102]:
def request_token_price(request):
    response = requests.get(request)
    response_json = response.json()
    return response_json['eu']

def new_clean_df():
    df = pd.DataFrame(request_token_price(JSON_URL))
    df['date']=df['time'].apply(lambda x: datetime.fromtimestamp(x))
    df['day_of_week']=df['date'].apply(lambda x: x.weekday())
    df['hour']=df['date'].apply(lambda x: x.hour)
    df['day']=df['date'].apply(lambda x: x.day)
    df['date']=pd.to_datetime(df['date'])
    df['month']=df['date'].apply(lambda x: x.month)
    df=df.set_index('date',drop=False)
    df = df.sort_index()
    return df

In [103]:
df=new_clean_df()
df

,price,time,date,day_of_week,hour,day,month
date,,,,,,,
2015-04-21 22:04:20,35000,1429646660,2015-04-21 22:04:20,1,22,21,4
2015-04-21 22:19:50,35350,1429647590,2015-04-21 22:19:50,1,22,21,4
2015-04-21 22:35:33,35704,1429648533,2015-04-21 22:35:33,1,22,21,4
2015-04-21 22:49:56,36061,1429649396,2015-04-21 22:49:56,1,22,21,4
2015-04-21 23:05:02,36422,1429650302,2015-04-21 23:05:02,1,23,21,4
...,...,...,...,...,...,...,...
2021-01-15 15:23:23,160569,1610720603,2021-01-15 15:23:23,4,15,15,1
2021-01-15 15:43:23,161550,1610721803,2021-01-15 15:43:23,4,15,15,1
2021-01-15 16:03:23,162004,1610723003,2021-01-15 16:03:23,4,16,15,1


In [104]:
df_small=pd.DataFrame.copy(df)

std = lambda x: (x.std())
df_small['7_day_std']=df['price'].rolling('7d').apply(std)
df_small['30_day_std']=df['price'].rolling('30d').apply(std)
df_small['7_day_mean']=df['price'].rolling('7d').mean()
df_small['30_day_mean']=df['price'].rolling('30d').mean()
df_small['1_day_mean']=df['price'].rolling('1d').mean()
df_small['7_day_low']=df['price'].rolling('7d').min()
df_small['30_day_low']=df['price'].rolling('30d').min()
df_small['7_day_high']=df['price'].rolling('7d').max()
df_small['30_day_high']=df['price'].rolling('30d').max()
df_small['1_day_high']=df['price'].rolling('1d').max()
df_small['1_day_low']=df['price'].rolling('1d').min()
df_small['last_change']=df['price'].diff(periods=2)
df_small=df_small.dropna()


In [105]:
df_small

,price,time,date,day_of_week,hour,day,month,7_day_std,30_day_std,7_day_mean,30_day_mean,1_day_mean,7_day_low,30_day_low,7_day_high,30_day_high,1_day_high,1_day_low,last_change
date,,,,,,,,,,,,,,,,,,,
2015-04-21 22:35:33,35704,1429648533,2015-04-21 22:35:33,1,22,21,4,352.001894,352.001894,35351.333333,35351.333333,35351.333333,35000.0,35000.0,35704.0,35704.0,35704.0,35000.0,704.0
2015-04-21 22:49:56,36061,1429649396,2015-04-21 22:49:56,1,22,21,4,456.629226,456.629226,35528.750000,35528.750000,35528.750000,35000.0,35000.0,36061.0,36061.0,36061.0,35000.0,711.0
2015-04-21 23:05:02,36422,1429650302,2015-04-21 23:05:02,1,23,21,4,562.104795,562.104795,35707.400000,35707.400000,35707.400000,35000.0,35000.0,36422.0,36422.0,36422.0,35000.0,718.0
2015-04-21 23:20:38,36786,1429651238,2015-04-21 23:20:38,1,23,21,4,668.330582,668.330582,35887.166667,35887.166667,35887.166667,35000.0,35000.0,36786.0,36786.0,36786.0,35000.0,725.0
2015-04-21 23:35:11,37154,1429652111,2015-04-21 23:35:11,1,23,21,4,775.556666,775.556666,36068.142857,36068.142857,36068.142857,35000.0,35000.0,37154.0,37154.0,37154.0,35000.0,732.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-01-15 15:23:23,160569,1610720603,2021-01-15 15:23:23,4,15,15,1,3050.854271,2887.161845,160933.333333,160442.181572,160505.222222,155955.0,155921.0,173013.0,173013.0,163376.0,158573.0,1826.0
2021-01-15 15:43:23,161550,1610721803,2021-01-15 15:43:23,4,15,15,1,3047.524487,2887.025150,160941.005952,160444.489612,160546.569444,155955.0,155921.0,173013.0,173013.0,163376.0,158670.0,1943.0
2021-01-15 16:03:23,162004,1610723003,2021-01-15 16:03:23,4,16,15,1,3043.972858,2887.206252,160949.980159,160446.915989,160585.583333,155955.0,155921.0,173013.0,173013.0,163376.0,158670.0,1435.0


In [109]:
df_day=pd.DataFrame.copy(df)
df_day.date=df_day.date.dt.floor('d')
df_day = df_day.reset_index(drop=True)
df_day = df_day.groupby(by=['date']).mean()
df_day=df_day.drop(columns=['hour','time'])
df_day['last_change']=df_day['price'].diff(periods=2)
df_day['7_day_low']=df_day['price'].rolling('7d').min()
df_day['30_day_low']=df_day['price'].rolling('30d').min()
df_day['7_day_high']=df_day['price'].rolling('7d').max()
df_day['30_day_high']=df_day['price'].rolling('30d').max()
df_day['7_day_mean']=df_day['price'].rolling('7d').mean()
df_day['30_day_mean']=df_day['price'].rolling('30d').mean()
df_day=df_day.dropna()
df_day['date']=df_day.index
df_day['day_of_week']=df_day['date'].apply(lambda x: x.weekday())
df_day = df_day.reset_index(drop=True)
df_day

,price,day_of_week,day,month,last_change,7_day_low,30_day_low,7_day_high,30_day_high,7_day_mean,30_day_mean,date
0,34058.500000,3,23.0,4.0,-2191.875000,34058.500000,34058.500000,41571.255319,41571.255319,37293.376773,37293.376773,2015-04-23
1,37422.458333,4,24.0,4.0,-4148.796986,34058.500000,34058.500000,41571.255319,41571.255319,37325.647163,37325.647163,2015-04-24
2,34447.492537,5,25.0,4.0,388.992537,34058.500000,34058.500000,41571.255319,41571.255319,36750.016238,36750.016238,2015-04-25
3,38888.300000,6,26.0,4.0,1465.841667,34058.500000,34058.500000,41571.255319,41571.255319,37106.396865,37106.396865,2015-04-26
4,39143.232877,0,27.0,4.0,4695.740339,34058.500000,34058.500000,41571.255319,41571.255319,37397.373438,37397.373438,2015-04-27
...,...,...,...,...,...,...,...,...,...,...,...,...
2043,162693.583333,0,11.0,1.0,4157.791667,158357.194444,158357.194444,162693.583333,162990.277778,159906.607794,160433.187024,2021-01-11
2044,163558.319444,1,12.0,1.0,3112.541667,158357.194444,158357.194444,163558.319444,163558.319444,160095.264540,160571.940496,2021-01-12
2045,161020.366197,2,13.0,1.0,-1673.217136,158357.194444,158357.194444,163558.319444,163558.319444,160449.582160,160448.821022,2021-01-13
2046,160985.666667,3,14.0,1.0,-2572.652778,158535.791667,158357.194444,163558.319444,163558.319444,160825.078191,160420.176925,2021-01-14


In [107]:
today=str(datetime.date(datetime.now()))
df_small.to_csv("Token-prices-small"+today+".csv",index=False)
df_day.to_csv("Token-prices-day"+today+".csv",index=False)